### Tutorial Brian2 para Izhikevich - 29.7.25

Aspectos que Brian2 maneja elegantemente:

- Ecuaciones diferenciales del modelo neuronal
- Condiciones de reset post-spike
- Conectividad espacialmente estructurada
- Monitoreo de spikes y variables de estado
- Paralelización automática

Ventajas clave:

- Soporte nativo para modelos de Izhikevich
- Manejo automático de delays axonales
- STDP implementado como funcionalidad built-in
- Sistema de unidades físicas integrado (ms, mV, etc.)
- Optimización automática del código generado

El framework te permitirá enfocarte en la lógica del modelo sin preocuparte por optimizaciones de bajo nivel. La sintaxis declarativa hace el código muy legible y modificable.

-----

#### Conceptos fundamentales de Brian2

Objetos principales:

- NeuronGroup: Poblaciones de neuronas con ecuaciones diferenciales
- Synapses: Conexiones sinápticas con plasticidad
- SpikeMonitor/StateMonitor: Registro de actividad
- run(): Ejecuta la simulación

Flujo típico:

- Definir ecuaciones neuronales
- Crear grupos de neuronas
- Establecer conectividad
- Configurar monitoreo
- Ejecutar y analizar

-----

Estrategia de implementación progresiva:

1. Red pequeña inicial (1000-5000 neuronas) para debugging rápido
2. Modelo neuronal básico primero, sin plasticidad
3. Conectividad simple con probabilidades fijas
4. Añadir delays de conducción axonal
5. Monitoreo y visualización en cada paso
6. .... Implementar STDP gradualmente
   
-----

-  #### Lección 1 - Simular una neurona básica de Izhikevich

- Implementar las ecuaciones básicas (dv/dt, du/dt)
- Condición de reset post-spike
- Probar con corriente constante
- Objetivo: Entender la sintaxis de ecuaciones en Brian2

In [ ]:
from brian2 import *

# Configuración inicial
start_scope()

# Ecuaciones del modelo de Izhikevich
equations = '''
dv/dt = (0.04*v**2 + 5*v + 140 - u + I)/ms : 1
du/dt = a*(b*v - u)/ms : 1
I : 1
a : 1
b : 1  
c : 1
d : 1
'''

# Crear neurona individual
neuron = NeuronGroup(1, equations, 
                     threshold='v >= 30',
                     reset='v = c; u += d',
                     method='euler')

# Parámetros para neurona Regular Spiking (RS)
neuron.a = 0.02
neuron.b = 0.2
neuron.c = -65
neuron.d = 8

# Condiciones iniciales
neuron.v = -65
neuron.u = neuron.b * neuron.v

# Input constante
neuron.I = 10

# Monitoreo
state_mon = StateMonitor(neuron, ['v', 'u'], record=True)
spike_mon = SpikeMonitor(neuron)


# Simulación
run(1000*ms)

# Visualización
figure(figsize=(12, 4))

subplot(121)
plot(state_mon.t/ms, state_mon.v[0])
xlabel('Tiempo (ms)')
ylabel('Potencial (mV)')
title('Potencial de membrana')
grid(True)

subplot(122)
plot(state_mon.t/ms, state_mon.u[0])
xlabel('Tiempo (ms)')
ylabel('Variable u')
title('Variable de recuperación')
grid(True)

tight_layout()
show()

print(f"Número de spikes: {len(spike_mon.t)}")
print(f"Frecuencia promedio: {len(spike_mon.t)/(1000*ms):.1f} Hz")

Sintaxis Brian2:

    - start_scope(): Limpia namespace anterior
    - equations = '''...''': String multilinea con ecuaciones
    - dv/dt = .../ms : 1: Unidades físicas obligatorias
    - method='euler': Integrador numérico

Conceptos clave:

    - Equations: Define las ecuaciones diferenciales como strings
    - Variables de estado: v (potencial), u (recuperación)
    - Parámetros: a, b, c, d
    - threshold y reset: Condiciones de spike

Parámetros para neurona RS (Regular Spiking):

    a = 0.02, b = 0.2, c = -65, d = 8

Input de prueba:

    Corriente constante: neuron.I = 10
    O input Poisson para ruido

Monitoreo:

    StateMonitor para v, u
    SpikeMonitor para timing

-----

- #### Lección 2 - Heterogeneidad Neuronal - Tipos de Izhikevich

Objetivo
- Crear diferentes tipos de neuronas (RS, IB, CH, FS, LTS) y observar sus patrones únicos de disparo.

Conceptos Brian2 Nuevos

- Parámetros heterogéneos: Asignar valores diferentes a cada neurona
- Indexing: Acceder a neuronas específicas neuron[0:4]
- Distribuciones: rand(), randn() para variabilidad

Tipos Neuronales del Paper

    RS: Adaptación de frecuencia, spikes regulares
    IB: Burst inicial, luego spikes regulares
    CH: Bursts repetitivos de alta frecuencia
    FS: Spikes rápidos sin adaptación
    LTS: Umbral bajo, adaptación moderada

Observaciones Esperadas

    RS: Pocos spikes iniciales, luego espaciados
    IB: Burst al inicio, después regular
    CH: Bursts repetitivos cada ~25ms
    FS: Tren continuo de spikes rápidos
    LTS: Similar a FS pero con ligera adaptación

In [ ]:
from brian2 import *
import numpy as np

# Configuración inicial
start_scope()

# Ecuaciones del modelo de Izhikevich
equations = '''
dv/dt = (0.04*v**2 + 5*v + 140 - u + I)/ms : 1
du/dt = a*(b*v - u)/ms : 1
I : 1
a : 1
b : 1  
c : 1
d : 1
'''

# Crear 5 neuronas (una de cada tipo)
neurons = NeuronGroup(5, equations, 
                      threshold='v >= 30',
                      reset='v = c; u += d',
                      method='euler')

# Parámetros para diferentes tipos neuronales
# Índice 0: RS (Regular Spiking)
neurons.a[0] = 0.02
neurons.b[0] = 0.2
neurons.c[0] = -65
neurons.d[0] = 8

# Índice 1: IB (Intrinsically Bursting)  
neurons.a[1] = 0.02
neurons.b[1] = 0.2
neurons.c[1] = -55
neurons.d[1] = 4

# Índice 2: CH (Chattering)
neurons.a[2] = 0.02
neurons.b[2] = 0.2
neurons.c[2] = -50
neurons.d[2] = 2

# Índice 3: FS (Fast Spiking) 
neurons.a[3] = 0.1
neurons.b[3] = 0.2
neurons.c[3] = -65
neurons.d[3] = 2

# Índice 4: LTS (Low Threshold Spiking)
neurons.a[4] = 0.02
neurons.b[4] = 0.25
neurons.c[4] = -65
neurons.d[4] = 2

# Condiciones iniciales
neurons.v = -65
neurons.u = neurons.b * neurons.v

# Input constante para todas
neurons.I = 10

# Monitoreo
state_mon = StateMonitor(neurons, ['v'], record=True)
spike_mon = SpikeMonitor(neurons)

# Simulación
run(1000*ms)

# Visualización
figure(figsize=(15, 10))

# Tipos de neuronas
types = ['RS (Regular)', 'IB (Bursting)', 'CH (Chattering)', 
         'FS (Fast)', 'LTS (Low Threshold)']
colors = ['blue', 'red', 'green', 'orange', 'purple']

# Plot individual para cada tipo
for i in range(5):
    subplot(3, 2, i+1)
    plot(state_mon.t/ms, state_mon.v[i], color=colors[i], linewidth=1.5)
    xlabel('Tiempo (ms)')
    ylabel('Potencial (mV)')
    title(f'{types[i]}')
    grid(True)
    ylim(-75, 35)

# Raster plot conjunto
subplot(3, 2, 6)
for i in range(5):
    spike_times = spike_mon.t[spike_mon.i == i]
    plot(spike_times/ms, [i]*len(spike_times), '|', 
         markersize=10, color=colors[i], label=types[i])

xlabel('Tiempo (ms)')
ylabel('Neurona')
title('Raster de Spikes')
legend(bbox_to_anchor=(1.05, 1), loc='upper left')
grid(True)

tight_layout()
show()

# Análisis de frecuencias
print("Análisis de frecuencias de disparo:")
print("-" * 40)
for i in range(5):
    spike_count = sum(spike_mon.i == i)
    frequency = spike_count / (1000*ms)
    print(f"{types[i]:15}: {spike_count:2d} spikes, {frequency:.1f} Hz")

-----

- #### Lección 3 - Simular población heterogénea con input estocástico (noisy driving)

Objetivo

- Crear una red de 100 neuronas con variabilidad realista y input estocástico, observando comportamiento colectivo.
  
Conceptos Brian2 Nuevos

- PoissonInput: Input estocástico realista
- Distribuciones aleatorias: rand(), randn()
- Arrays numpy: Asignación vectorial de parámetros
- Análisis de red: Frecuencias, correlaciones

```
# Input estocástico independiente por neurona
PoissonInput(target_group, target_variable, 
             N=spikes_per_second, rate=frequency, weight=strength)
```

In [ ]:
from brian2 import *
import numpy as np
import matplotlib.pyplot as plt

# Configuración inicial
start_scope()

# Parámetros de la red
N_exc = 80  # Neuronas excitatorias
N_inh = 20  # Neuronas inhibitorias  
N_total = N_exc + N_inh

# Ecuaciones del modelo de Izhikevich
equations = '''
dv/dt = (0.04*v**2 + 5*v + 140 - u + I_noise)/ms : 1
du/dt = a*(b*v - u)/ms : 1
I_noise : 1  # Corriente de ruido externa
a : 1
b : 1  
c : 1
d : 1
'''

# Crear población de neuronas
neurons = NeuronGroup(N_total, equations, 
                      threshold='v >= 30',
                      reset='v = c; u += d',
                      method='euler')

# Heterogeneidad en parámetros (como en el paper original)
# Variables aleatorias para cada neurona
r_exc = np.random.rand(N_exc)
r_inh = np.random.rand(N_inh)

# Neuronas excitatorias (0 a 79)
neurons.a[0:N_exc] = 0.02
neurons.b[0:N_exc] = 0.2
neurons.c[0:N_exc] = -65 + 15 * r_exc**2  # Rango: -65 a -50
neurons.d[0:N_exc] = 8 - 6 * r_exc**2     # Rango: 8 a 2

# Neuronas inhibitorias (80 a 99)  
neurons.a[N_exc:] = 0.02 + 0.08 * r_inh   # Rango: 0.02 a 0.1
neurons.b[N_exc:] = 0.25 - 0.05 * r_inh   # Rango: 0.25 a 0.2
neurons.c[N_exc:] = -65
neurons.d[N_exc:] = 2

# Condiciones iniciales
neurons.v = -65
neurons.u = neurons.b * neurons.v

# Input estocástico (simula input talámico)
# Cada neurona recibe spikes Poisson independientes
poisson_exc = PoissonInput(neurons[0:N_exc], 'I_noise', 
                          N=80, rate=1.5*Hz, weight=0.1)
poisson_inh = PoissonInput(neurons[N_exc:], 'I_noise', 
                          N=20, rate=1*Hz, weight=0.08)

# Monitoreo
state_mon = StateMonitor(neurons, ['v'], record=[0, 1, 2, 80, 81])
spike_mon = SpikeMonitor(neurons)

# Simulación
print("Ejecutando simulación de 2 segundos...")
run(2000*ms)

# Análisis y visualización
figure(figsize=(15, 12))

# 1. Raster plot de toda la población
subplot(3, 2, 1)
plot(spike_mon.t/ms, spike_mon.i, '.k', markersize=1)
axhline(y=N_exc-0.5, color='red', linestyle='--', alpha=0.7)
xlabel('Tiempo (ms)')
ylabel('Neurona #')
title('Raster Plot (línea roja separa exc/inh)')
xlim(0, 2000)

# 2. Zoom del raster (primeros 500ms)
subplot(3, 2, 2)
mask = spike_mon.t < 500*ms
plot(spike_mon.t[mask]/ms, spike_mon.i[mask], '.k', markersize=2)
axhline(y=N_exc-0.5, color='red', linestyle='--', alpha=0.7)
xlabel('Tiempo (ms)')
ylabel('Neurona #')
title('Raster Plot - Zoom (0-500ms)')

# 3. Trazas de voltaje representativas
subplot(3, 2, 3)
colors = ['blue', 'blue', 'blue', 'red', 'red']
labels = ['Exc 1', 'Exc 2', 'Exc 3', 'Inh 1', 'Inh 2']
for i, (color, label) in enumerate(zip(colors, labels)):
    plot(state_mon.t/ms, state_mon.v[i] + i*80, 
         color=color, label=label, linewidth=0.8)
xlabel('Tiempo (ms)')
ylabel('Potencial + offset (mV)')
title('Trazas de Voltaje')
legend()
xlim(0, 500)

# 4. Histograma de frecuencias por neurona
subplot(3, 2, 4)
freq_exc = []
freq_inh = []

for i in range(N_total):
    spike_count = sum(spike_mon.i == i)
    freq = spike_count / (2000*ms)  # Hz
    if i < N_exc:
        freq_exc.append(freq)
    else:
        freq_inh.append(freq)

hist(freq_exc, bins=15, alpha=0.7, color='blue', label=f'Excitatorias (n={N_exc})')
hist(freq_inh, bins=15, alpha=0.7, color='red', label=f'Inhibitorias (n={N_inh})')
xlabel('Frecuencia (Hz)')
ylabel('Número de neuronas')
title('Distribución de Frecuencias')
legend()

# 5. Actividad poblacional en ventanas de tiempo
subplot(3, 2, 5)
bin_size = 50*ms
time_bins = np.arange(0, 2000, bin_size/ms)
pop_activity = []

for t in time_bins:
    spikes_in_bin = sum((spike_mon.t >= t*ms) & (spike_mon.t < (t + bin_size/ms)*ms))
    pop_activity.append(spikes_in_bin)

plot(time_bins + bin_size/(2*ms), pop_activity, 'g-', linewidth=2)
xlabel('Tiempo (ms)')
ylabel(f'Spikes por {bin_size/ms:.0f}ms')
title('Actividad Poblacional')
grid(True)

# 6. Distribución de parámetros c y d
subplot(3, 2, 6)
scatter(neurons.c[0:N_exc], neurons.d[0:N_exc], 
        c='blue', alpha=0.6, label='Excitatorias', s=30)
scatter(neurons.c[N_exc:], neurons.d[N_exc:], 
        c='red', alpha=0.6, label='Inhibitorias', s=30)
xlabel('Parámetro c (mV)')
ylabel('Parámetro d')
title('Distribución de Parámetros')
legend()
grid(True)

tight_layout()
show()

# Estadísticas de la simulación
print("\n" + "="*50)
print("ESTADÍSTICAS DE LA SIMULACIÓN")
print("="*50)
print(f"Neuronas totales: {N_total}")
print(f"Spikes totales: {len(spike_mon.t)}")
print(f"Duración: 2000 ms")
print()
print("Frecuencias promedio:")
print(f"  Excitatorias: {np.mean(freq_exc):.1f} ± {np.std(freq_exc):.1f} Hz")
print(f"  Inhibitorias: {np.mean(freq_inh):.1f} ± {np.std(freq_inh):.1f} Hz")
print(f"  Red completa: {len(spike_mon.t)/(2000*ms*N_total):.1f} Hz")
print()
print("Neuronas activas:")
print(f"  Excitatorias: {sum(np.array(freq_exc) > 0)}/{N_exc} ({100*sum(np.array(freq_exc) > 0)/N_exc:.1f}%)")
print(f"  Inhibitorias: {sum(np.array(freq_inh) > 0)}/{N_inh} ({100*sum(np.array(freq_inh) > 0)/N_inh:.1f}%)")

Heterogeneidad Realista

- Distribución de parámetros como en el paper original
- Excitatorias: Continuo RS → CH mediante r²
- Inhibitorias: Continuo FS → LTS mediante r

Análisis de Red Neuronal

- Raster plots: Visualizar patrones temporales
- Histogramas de frecuencia: Distribución de actividad
- Actividad poblacional: Dinámicas colectivas

Observaciones Esperadas

- Actividad asíncrona: Sin sincronización global
- Distribución heterogénea: Diferentes frecuencias por neurona
- Excitatorias más activas: Mayor frecuencia promedio
- Irregularidad temporal: Actividad tipo Poisson

------

- #### Lección 4 - Población heterogénea con conexiones sinápticas + input externo


Objetivo
- Introducir sinapsis reales entre neuronas con probabilidad de conexión y delays axonales.

Conceptos Brian2 Nuevos

- Synapses: Conexiones sinápticas entre grupos
- connect(): Patrones de conectividad
- delay: Delays de conducción axonal

Synapses en Brian2
```
# Objeto sináptico básico
syn = Synapses(source_group, target_group,
               'w : 1',  # Variables sinápticas
               on_pre='I_syn_post += w')  # Acción al recibir spike
```

In [ ]:
from brian2 import *
import numpy as np
import matplotlib.pyplot as plt

start_scope()

# Parámetros de la red
N_exc = 40  # Reducido para visualización clara
N_inh = 10
N_total = N_exc + N_inh

# Ecuaciones con corriente sináptica
equations = '''
dv/dt = (0.04*v**2 + 5*v + 140 - u + I_syn + I_noise)/ms : 1
du/dt = a*(b*v - u)/ms : 1
I_syn : 1  # Corriente sináptica total
I_noise : 1  # Input externo
a : 1
b : 1  
c : 1
d : 1
'''

# Crear grupos separados para mejor control
exc_neurons = NeuronGroup(N_exc, equations, 
                          threshold='v >= 30',
                          reset='v = c; u += d',
                          method='euler')

inh_neurons = NeuronGroup(N_inh, equations,
                          threshold='v >= 30', 
                          reset='v = c; u += d',
                          method='euler')

# Parámetros heterogéneos
r_exc = np.random.rand(N_exc)
r_inh = np.random.rand(N_inh)

# Excitatorias: RS principalmente
exc_neurons.a = 0.02
exc_neurons.b = 0.2
exc_neurons.c = -65 + 10 * r_exc**2
exc_neurons.d = 8 - 4 * r_exc**2

# Inhibitorias: FS principalmente  
inh_neurons.a = 0.1
inh_neurons.b = 0.2
inh_neurons.c = -65
inh_neurons.d = 2

# Condiciones iniciales
exc_neurons.v = -65
exc_neurons.u = exc_neurons.b * exc_neurons.v
inh_neurons.v = -65
inh_neurons.u = inh_neurons.b * inh_neurons.v

# SINAPSIS - Aquí está la novedad principal
# 1. Excitatorias -> Excitatorias
syn_ee = Synapses(exc_neurons, exc_neurons,
                  'w : 1',
                  on_pre='I_syn_post += w')
syn_ee.connect(p=0.1)  # 10% probabilidad de conexión
syn_ee.w = 0.5
syn_ee.delay = 'rand() * 5*ms'  # Delays 0-5ms

# 2. Excitatorias -> Inhibitorias
syn_ei = Synapses(exc_neurons, inh_neurons,
                  'w : 1', 
                  on_pre='I_syn_post += w')
syn_ei.connect(p=0.2)  # Mayor probabilidad exc->inh
syn_ei.w = 1.0  # Más fuerte
syn_ei.delay = '1*ms + rand() * 2*ms'

# 3. Inhibitorias -> Excitatorias  
syn_ie = Synapses(inh_neurons, exc_neurons,
                  'w : 1',
                  on_pre='I_syn_post -= w')  # Negativo = inhibición
syn_ie.connect(p=0.3)  # Alta conectividad inhibitoria
syn_ie.w = 2.0  # Inhibición fuerte
syn_ie.delay = '0.5*ms + rand() * 1*ms'

# 4. Inhibitorias -> Inhibitorias
syn_ii = Synapses(inh_neurons, inh_neurons,
                  'w : 1',
                  on_pre='I_syn_post -= w')
syn_ii.connect(p=0.1)
syn_ii.w = 1.0
syn_ii.delay = '1*ms'

# Input externo reducido (ahora la red se auto-sustenta)

poisson_exc = PoissonInput(exc_neurons, 'I_noise', N=40, rate=1.5*Hz, weight=0.08)
poisson_inh = PoissonInput(inh_neurons, 'I_noise', N=10, rate=1*Hz, weight=0.04)

# Monitoreo
spike_mon_exc = SpikeMonitor(exc_neurons)
spike_mon_inh = SpikeMonitor(inh_neurons)
state_mon_exc = StateMonitor(exc_neurons, ['v', 'I_syn'], record=[0, 1, 2])
state_mon_inh = StateMonitor(inh_neurons, ['v', 'I_syn'], record=[0, 1])

print("Simulando red con sinapsis...")
run(1500*ms)

# Visualización
figure(figsize=(15, 12))

# 1. Raster plot conjunto
subplot(3, 3, 1)
plot(spike_mon_exc.t/ms, spike_mon_exc.i, '.b', markersize=3, label='Excitatorias')
plot(spike_mon_inh.t/ms, spike_mon_inh.i + N_exc, '.r', markersize=4, label='Inhibitorias')
axhline(y=N_exc-0.5, color='k', linestyle='--', alpha=0.5)
xlabel('Tiempo (ms)')
ylabel('Neurona #')
title('Raster Plot - Red Conectada')
legend()

# 2. Zoom raster (500-1000ms)
subplot(3, 3, 2)
mask_exc = (spike_mon_exc.t >= 500*ms) & (spike_mon_exc.t < 1000*ms)
mask_inh = (spike_mon_inh.t >= 500*ms) & (spike_mon_inh.t < 1000*ms)
plot(spike_mon_exc.t[mask_exc]/ms, spike_mon_exc.i[mask_exc], '.b', markersize=4)
plot(spike_mon_inh.t[mask_inh]/ms, spike_mon_inh.i[mask_inh] + N_exc, '.r', markersize=5)
xlabel('Tiempo (ms)')
ylabel('Neurona #')
title('Zoom 500-1000ms')

# 3. Voltajes con corrientes sinápticas
subplot(3, 3, 3)
plot(state_mon_exc.t/ms, state_mon_exc.v[0], 'b-', label='V exc', linewidth=1)
plot(state_mon_exc.t/ms, state_mon_exc.I_syn[0]*10 - 50, 'g-', 
     label='I_syn×10 - 50', linewidth=1, alpha=0.7)
xlim(200, 400)
xlabel('Tiempo (ms)')
ylabel('mV / corriente')
title('Voltaje + Corriente Sináptica')
legend()

# 4. Distribución de grados de conectividad
subplot(3, 3, 4)
# Contar conexiones por neurona
conn_in_exc = np.zeros(N_exc)
conn_out_exc = np.zeros(N_exc)

for i in range(len(syn_ee.i)):
    conn_out_exc[syn_ee.i[i]] += 1
    conn_in_exc[syn_ee.j[i]] += 1

hist(conn_in_exc, bins=8, alpha=0.7, label='Conexiones entrantes')
hist(conn_out_exc, bins=8, alpha=0.7, label='Conexiones salientes')
xlabel('Número de conexiones')
ylabel('Neuronas')
title('Grado de Conectividad Exc-Exc')
legend()

# 5. Actividad poblacional
subplot(3, 3, 5)
bin_size = 20*ms
time_bins = np.arange(0, 1500, bin_size/ms)
activity_exc = []
activity_inh = []

for t in time_bins:
    exc_spikes = sum((spike_mon_exc.t >= t*ms) & (spike_mon_exc.t < (t + bin_size/ms)*ms))
    inh_spikes = sum((spike_mon_inh.t >= t*ms) & (spike_mon_inh.t < (t + bin_size/ms)*ms))
    activity_exc.append(exc_spikes)
    activity_inh.append(inh_spikes)

plot(time_bins, activity_exc, 'b-', label='Excitatorias', linewidth=2)
plot(time_bins, activity_inh, 'r-', label='Inhibitorias', linewidth=2)
xlabel('Tiempo (ms)')
ylabel(f'Spikes/{bin_size/ms:.0f}ms')
title('Actividad Poblacional')
legend()
grid(True)

# 6. Correlación E-I
subplot(3, 3, 6)
scatter(activity_exc, activity_inh, alpha=0.6)
xlabel('Actividad Excitatorias')
ylabel('Actividad Inhibitorias')
title('Correlación E-I')
grid(True)

# 7. Distribución de delays
subplot(3, 3, 7)
hist(syn_ee.delay/ms, bins=15, alpha=0.7, label='Exc-Exc')
hist(syn_ei.delay/ms, bins=15, alpha=0.7, label='Exc-Inh') 
hist(syn_ie.delay/ms, bins=15, alpha=0.7, label='Inh-Exc')
xlabel('Delay (ms)')
ylabel('Número de sinapsis')
title('Distribución de Delays')
legend()

# 8. Matriz de conectividad (muestra)
subplot(3, 3, 8)
conn_matrix = np.zeros((15, 15))  # Solo primeras 15 neuronas
for i in range(len(syn_ee.i)):
    if syn_ee.i[i] < 15 and syn_ee.j[i] < 15:
        conn_matrix[syn_ee.i[i], syn_ee.j[i]] = syn_ee.w[i]

imshow(conn_matrix, cmap='Blues', aspect='equal')
xlabel('Neurona post')
ylabel('Neurona pre')
title('Matriz Conectividad (15×15)')
colorbar()

# 9. Estadísticas de red
subplot(3, 3, 9)
text(0.1, 0.8, f'Neuronas: {N_exc}E + {N_inh}I', transform=gca().transAxes, fontsize=10)
text(0.1, 0.7, f'Sinapsis E-E: {len(syn_ee.i)}', transform=gca().transAxes, fontsize=10)
text(0.1, 0.6, f'Sinapsis E-I: {len(syn_ei.i)}', transform=gca().transAxes, fontsize=10)
text(0.1, 0.5, f'Sinapsis I-E: {len(syn_ie.i)}', transform=gca().transAxes, fontsize=10)
text(0.1, 0.4, f'Sinapsis I-I: {len(syn_ii.i)}', transform=gca().transAxes, fontsize=10)

spikes_exc = len(spike_mon_exc.t)
spikes_inh = len(spike_mon_inh.t)
freq_exc = spikes_exc / (1.5 * N_exc)
freq_inh = spikes_inh / (1.5 * N_inh)

text(0.1, 0.25, f'Freq Exc: {freq_exc:.1f} Hz', transform=gca().transAxes, fontsize=10)
text(0.1, 0.15, f'Freq Inh: {freq_inh:.1f} Hz', transform=gca().transAxes, fontsize=10)
axis('off')
title('Estadísticas de Red')

tight_layout()
show()

print(f"\nConexiones creadas:")
print(f"  E→E: {len(syn_ee.i)} ({len(syn_ee.i)/(N_exc*N_exc)*100:.1f}%)")
print(f"  E→I: {len(syn_ei.i)} ({len(syn_ei.i)/(N_exc*N_inh)*100:.1f}%)")
print(f"  I→E: {len(syn_ie.i)} ({len(syn_ie.i)/(N_inh*N_exc)*100:.1f}%)")
print(f"  I→I: {len(syn_ii.i)} ({len(syn_ii.i)/(N_inh*N_inh)*100:.1f}%)")
print(f"\nFrecuencias: Exc {freq_exc:.1f} Hz, Inh {freq_inh:.1f} Hz")

Tipos de Conectividad

- E→E: Excitación recurrente (p=0.1, débil)
- E→I: Activación inhibitoria (p=0.2, fuerte)
- I→E: Inhibición feedback (p=0.3, muy fuerte)
- I→I: Inhibición lateral (p=0.1, moderada)

Delays Axonales

- E→E: 0-5ms (conexiones locales variables)
- I→E: 0.5-1.5ms (inhibición rápida)

Comportamiento Emergente Esperado

- Balance excitación/inhibición automático
- Frecuencias realistas (~5-15 Hz)
- Correlaciones E-I por retroalimentación
- Irregularidad temporal por competencia

Mejoras logradas:

- Actividad real: 1.8 Hz exc, 7.1 Hz inh (frecuencias biológicas)
- Balance E-I: Inhibitorias más activas (correcto)
- Dinámicas emergentes: Actividad irregular post-600ms
- Conectividad funcional: ~10% E-E, ~20% E-I como esperado

Observaciones clave:

- Período inicial silencioso, luego actividad sostenida
- Correlación E-I positiva (retroalimentación)
- Distribución realista de delays (1-2ms pico)
- Matriz de conectividad sparse apropiada
-----